<a href="https://colab.research.google.com/github/ZilferII/CodeVault/blob/main/AutoGPT_stocksv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y build-essential wget libncurses5-dev libncursesw5-dev
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,566 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,974 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,398 kB]
Hit:13 https://ppa.launchpadcontent.

In [ ]:
!pip install alpaca-trade-api alpha_vantage tensorflow scikit-learn matplotlib dash dash-core-components dash-html-components dash-bootstrap-components fastapi uvicorn nest-asyncio pyngrok transformers plotly pandas requests backtrader boto3 keras-tuner deap

In [ ]:
import os
os.environ['ALPACA_API_KEY'] = 'PKDEDZT5O8CI3R6UTONM'
os.environ['ALPACA_SECRET_KEY'] = 'aPHjNjSp7d6gfDeUY1aElMc92iijlvcMUrnlTIUr'
os.environ['ALPACA_ENDPOINT'] = 'https://paper-api.alpaca.markets'
os.environ['NEWS_API_KEY'] = '8cc36860da3d4624810890fe4feb78e3'
os.environ['FMP_API_KEY'] = 'efCfJ6ItVI48a16QgQrc8WxH4BORgQTa'

In [ ]:
import os

# Define the path for the ngrok configuration file
xdg_config_path = os.path.expanduser("~/.config/ngrok/ngrok.yml")

# Define the content of the configuration file
ngrok_config_content = """
authtoken: 2iXBVyuqMaMjHyK44ixjAcouHKe_5A2z1GqFXjoaST4Yh7GDQ
tunnels:
  myservice:
    addr: 8000
    proto: http
"""

# Ensure the directory exists and write the configuration file
config_dir = os.path.dirname(xdg_config_path)
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
with open(xdg_config_path, 'w') as config_file:
    config_file.write(ngrok_config_content)

print(f"ngrok configuration file created at {xdg_config_path}")

In [ ]:
!ngrok config check

In [ ]:
%%writefile data_streams.py
import os
import requests
import pandas as pd
from queue import Queue
from threading import Thread
import time

ALPACA_API_KEY = os.getenv('ALPACA_API_KEY')
ALPACA_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')
ALPACA_ENDPOINT = os.getenv('ALPACA_ENDPOINT', 'https://paper-api.alpaca.markets')
FMP_API_KEY = os.getenv('FMP_API_KEY')

data_queue = Queue()

def fetch_data(endpoint, params={}):
    base_url = f"https://financialmodelingprep.com/api"
    url = f"{base_url}{endpoint}?apikey={FMP_API_KEY}"
    for key, value in params.items():
        url += f"&{key}={value}"
    response = requests.get(url)
    return response.json()

def normalize_data(data):
    if isinstance(data, list):
        return pd.DataFrame(data)
    elif isinstance(data, dict) and 'data' in data:
        return pd.DataFrame(data['data'])
    return pd.DataFrame()

def fetch_alpaca_data(symbol):
    import alpaca_trade_api as tradeapi
    api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPACA_ENDPOINT, api_version='v2')
    bars = api.get_bars(symbol, '1D', limit=100)
    return [{'time': bar.t.isoformat(), 'open': bar.o, 'high': bar.h, 'low': bar.l, 'close': bar.c, 'volume': bar.v} for bar in bars]

def fetch_real_time_data():
    while True:
        combined_data = {
            "stock_quote": fetch_data('/v3/quote/AAPL'),
            "historical_prices": fetch_alpaca_data('AAPL'),
            "news": fetch_data('/v3/stock_news', {'tickers': 'AAPL', 'limit': 50}),
            "social_sentiment": fetch_data('/v4/historical/social-sentiment', {'symbol': 'AAPL', 'page': 0}),
        }
        normalized_data = {key: normalize_data(value) for key, value in combined_data.items()}
        data_queue.put(normalized_data)
        time.sleep(60)

In [ ]:
%%writefile preprocessing.py
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from feature_engineering import extract_features, analyze_sentiments

def preprocess_data(data):
    features = extract_features(data)
    features = analyze_sentiments(features)

    historical_prices = pd.DataFrame(features['historical_prices'])
    social_sentiment = pd.DataFrame(features['social_sentiment'])

    scaler = MinMaxScaler(feature_range=(0, 1))
    historical_prices['scaled_close'] = scaler.fit_transform(historical_prices[['close']])

    merged_data = historical_prices.merge(social_sentiment, left_on='time', right_on='date', how='left')
    merged_data.fillna(method='ffill', inplace=True)
    return merged_data, scaler

def create_lstm_dataset(data, window_size=60):
    X, y = [], []
    for i in range(len(data) - window_size - 1):
        X.append(data[i:(i + window_size), 0])
        y.append(data[i + window_size, 0])
    return np.array(X), np.array(y)

def preprocess_for_random_forest(data):
    data = data.dropna()
    X = data.drop(columns=['target']).values
    y = data['target'].values
    return X, y

In [ ]:
%%writefile feature_engineering.py
import pandas as pd
from transformers import pipeline

def extract_features(data):
    features = []
    for key, df in data.items():
        if key == 'historical_prices':
            df['close'] = df['close'].astype(float)
            features.extend(df[['time', 'close']].to_dict('records'))
        elif key == 'social_sentiment':
            features.extend(df[['date', 'sentiment']].to_dict('records'))
    return features

def analyze_sentiments(features):
    sentiment_analysis = pipeline("sentiment-analysis")
    for feature in features:
        if 'sentiment' in feature:
            sentiment_result = sentiment_analysis(feature['sentiment'])[0]
            feature['sentiment_analysis'] = sentiment_result['label']
    return features

In [ ]:
%%writefile model_preparation.py
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(input_shape, 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_lstm_model(X_train, y_train, X_val, y_val):
    model = build_lstm_model(X_train.shape[1])
    model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), verbose=1)
    return model

def build_rl_model(input_shape):
    model = Sequential()
    model.add(Dense(24, input_dim=input_shape, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(2, activation='linear'))
    model.compile(loss='mse', optimizer='adam')
    return model

def train_rl_model(model, data, epochs=100):
    for epoch in range(epochs):
        state = data[0]
        for t in range(len(data) - 1):
            action = np.argmax(model.predict(state.reshape(1, -1)))
            next_state = data[t + 1]
            reward = get_reward(next_state, action)
            target = reward + 0.99 * np.max(model.predict(next_state.reshape(1, -1)))
            target_f = model.predict(state.reshape(1, -1))
            target_f[0][action] = target
            model.fit(state.reshape(1, -1), target_f, epochs=1, verbose=0)
            state = next_state

def get_reward(state, action):
    price_change = state[1] - state[0]
    if action == 1:  # Buy action
        return price_change if price_change > 0 else -1
    elif action == 0:  # Sell action
        return -price_change if price_change < 0 else -1
    return -1

In [ ]:
%%writefile pattern_recognition.py
import talib
import pandas as pd

def recognize_patterns(data):
    patterns = []
    open_prices = data['open'].values
    high_prices = data['high'].values
    low_prices = data['low'].values
    close_prices = data['close'].values

    engulfing = talib.CDLENGULFING(open_prices, high_prices, low_prices, close_prices)

    for i in range(len(engulfing)):
        if engulfing[i] == 100:  # Bullish engulfing
            patterns.append({
                'symbol': data['symbol'].iloc[i],
                'pattern': 'bullish_engulfing',
                'confidence': 0.9,
                'index': i
            })
        elif engulfing[i] == -100:  # Bearish engulfing
            patterns.append({
                'symbol': data['symbol'].iloc[i],
                'pattern': 'bearish_engulfing',
                'confidence': 0.9,
                'index': i
            })
    return patterns

In [ ]:
%%writefile trade_execution.py
import alpaca_trade_api as tradeapi
import os
import time

ALPACA_API_KEY = os.getenv('ALPACA_API_KEY')
ALPACA_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')
ALPACA_ENDPOINT = os.getenv('ALPACA_ENDPOINT', 'https://paper-api.alpaca.markets')

api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPACA_ENDPOINT, api_version='v2')

def execute_trade(action, symbol, qty):
    try:
        if action == 'buy':
            api.submit_order(
                symbol=symbol,
                qty=qty,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
        elif action == 'sell':
            api.submit_order(
                symbol=symbol,
                qty=qty,
                side='sell',
                type='market',
                time_in_force='gtc'
            )
        print(f"Executed {action} trade for {symbol} ({qty} shares).")
    except Exception as e:
        print(f"Trade execution failed: {e}")
        time.sleep(5)
        execute_trade(action, symbol, qty)  # Retry once after a short delay

In [ ]:
%%writefile visualization.py
from dash import dcc, html, Dash
from dash.dependencies import Input, Output, State
import pandas as pd
from threading import Thread
import plotly.graph_objs as go
from data_streams import data_queue
from trade_execution import execute_trade
from preprocessing import preprocess_data, create_lstm_dataset
from model_preparation import build_lstm_model, train_lstm_model

dash_app = Dash(__name__)

dash_app.layout = html.Div([
    html.H1("AutoGPT Trader Dashboard"),
    dcc.Dropdown(
        id='stock-selector',
        options=[{'label': 'AAPL', 'value': 'AAPL'}, {'label': 'GOOG', 'value': 'GOOG'}, {'label': 'TSLA', 'value': 'TSLA'}],
        value='AAPL'
    ),
    dcc.Dropdown(
        id='risk-level',
        options=[{'label': 'Low', 'value': 1}, {'label': 'Medium', 'value': 2}, {'label': 'High', 'value': 3}],
        value=2
    ),
    dcc.Graph(id='price-chart'),
    html.Button('Run Model', id='run-model', n_clicks=0),
    html.Button('Backtest', id='backtest-button', n_clicks=0),
    html.Button('Train Sharpe', id='sharpe-train-button', n_clicks=0),
    html.Div(id='trade-output'),
    html.Div(id='backtest-output'),
    html.Div(id='sharpe-output'),
    html.Div([
        html.Label('Alpaca API Key:'),
        dcc.Input(id='api-key', type='text', value=''),
        html.Label('Alpaca API Secret:'),
        dcc.Input(id='api-secret', type='password', value=''),
        html.Label('Alpaca Endpoint:'),
        dcc.Input(id='api-endpoint', type='text', value=''),
        html.Button('Update API Keys', id='update-api-button', n_clicks=0),
        html.Div(id='update-status')
    ])
])

@dash_app.callback(
    [Output('price-chart', 'figure'), Output('trade-output', 'children')],
    [Input('run-model', 'n_clicks'), Input('stock-selector', 'value'), Input('risk-level', 'value')]
)
def update_graph(n_clicks, stock, risk_level):
    if n_clicks > 0:
        if not data_queue.empty():
            data = data_queue.get()
            preprocessed_data, scaler = preprocess_data(data)
            X, y = create_lstm_dataset(preprocessed_data['scaled_close'].values.reshape(-1, 1))
            lstm_model = build_lstm_model(X.shape[1])
            lstm_model.fit(X, y, epochs=50, verbose=1)
            predictions = lstm_model.predict(X)
            trades = [{'symbol': stock, 'profit_margin': 0.05, 'risk': 50}]
            execute_trade('buy', stock, 10)  # Replace with actual trading logic
            fig = {
                'data': [{'x': preprocessed_data['time'], 'y': preprocessed_data['close'], 'type': 'line'}],
                'layout': {'title': f'Historical Prices for {stock}'}
            }
            return fig, "Trade executed based on model predictions and risk level"
        return {}, "No data available"
    return {}, "Click 'Run Model' to execute trading logic"

@dash_app.callback(
    Output('backtest-output', 'children'),
    [Input('backtest-button', 'n_clicks')],
    [Input('stock-selector', 'value')]
)
def backtest_strategy(n_clicks, stock):
    if n_clicks > 0:
        backtest_results = perform_backtest(stock, '2020-01-01', '2021-01-01')
        return f"Backtest Results for {stock}: {backtest_results}"
    return "Click 'Backtest' to run the strategy"

def perform_backtest(symbol, start_date, end_date):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(LSTMStrategy)
    historical_prices = fetch_historical_data(symbol, start_date, end_date)
    df = pd.DataFrame(historical_prices)
    data = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data)
    cerebro.broker.set_cash(100000)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
    results = cerebro.run()
    strat = results[0]
    sharpe_ratio = strat.analyzers.sharpe_ratio.get_analysis()['sharperatio']
    return f'Sharpe Ratio: {sharpe_ratio}'

@dash_app.callback(
    Output('sharpe-output', 'children'),
    [Input('sharpe-train-button', 'n_clicks')],
    [Input('stock-selector', 'value')]
)
def train_sharpe_strategy(n_clicks, stock):
    if n_clicks > 0:
        training_results = train_sharpe(stock)
        return f"Sharpe Training Results for {stock}: {training_results}"
    return "Click 'Train Sharpe' to start the training"

def train_sharpe(stock):
    historical_prices = fetch_historical_data(stock, '2020-01-01', '2021-01-01')
    df = pd.DataFrame(historical_prices)
    returns = df['close'].pct_change().dropna()
    mean_return = returns.mean()
    std_return = returns.std()
    sharpe_ratio = mean_return / std_return
    return f'Trained Sharpe Ratio: {sharpe_ratio}'

@dash_app.callback(
    Output('update-status', 'children'),
    [Input('update-api-button', 'n_clicks')],
    [State('api-key', 'value'), State('api-secret', 'value'), State('api-endpoint', 'value')]
)
def update_api_keys(n_clicks, new_api_key, new_api_secret, new_api_endpoint):
    if n_clicks > 0:
        os.environ['ALPACA_API_KEY'] = new_api_key
        os.environ['ALPACA_SECRET_KEY'] = new_api_secret
        os.environ['ALPACA_ENDPOINT'] = new_api_endpoint
        return "API keys updated successfully!"
    return ""

def within_risk_level(trade, risk_level):
    return trade['risk'] <= risk_level

def run_dash():
    dash_app.run_server(debug=True, use_reloader=False, port=8000)

In [ ]:
%%writefile backtesting.py
import pandas as pd
import backtrader as bt
from datetime import datetime
from historical_data import fetch_historical_data
from preprocessing import preprocess_data
from model_preparation import build_lstm_model

class LSTMStrategy(bt.Strategy):
    def __init__(self):
        self.lstm_model = build_lstm_model(input_shape=60)  # Adjust input shape as needed
        self.scaler = MinMaxScaler(feature_range=(0, 1))

    def next(self):
        data = self.data.close.get(size=60)  # Get the last 60 data points
        scaled_data = self.scaler.transform(np.array(data).reshape(-1, 1))
        X = np.array([scaled_data])
        prediction = self.lstm_model.predict(X)
        predicted_price = self.scaler.inverse_transform(prediction)[0][0]

        if predicted_price > self.data.close[0]:
            self.buy()
        else:
            self.sell()

def perform_backtest(symbol, start_date, end_date):
    data = fetch_historical_data(symbol, start_date, end_date)
    cerebro = bt.Cerebro()
    cerebro.addstrategy(LSTMStrategy)
    datafeed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(datafeed)
    cerebro.broker.set_cash(100000)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
    results = cerebro.run()
    strat = results[0]
    sharpe_ratio = strat.analyzers.sharpe_ratio.get_analysis()['sharperatio']
    return f'Sharpe Ratio: {sharpe_ratio}'

In [ ]:
%%writefile hyperparameter_visualization.py
import pandas as pd
import plotly.graph_objs as go
from dash import dcc, html, Input, Output
import dash

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H3('Hyperparameter Tuning Visualization'),
    dcc.Graph(id='hyperparam-tuning-graph'),
    html.Button('Train Model', id='train-model-button', n_clicks=0),
    html.Div(id='training-status')
])

@app.callback(
    Output('hyperparam-tuning-graph', 'figure'),
    Input('train-model-button', 'n_clicks'))
def update_hyperparam_graph(n_clicks):
    if n_clicks > 0:
        data = pd.DataFrame({
            'param': ['learning_rate', 'dropout', 'units', 'learning_rate', 'dropout', 'units'],
            'value': [0.01, 0.2, 50, 0.02, 0.3, 100],
            'loss': [0.5, 0.4, 0.3, 0.35, 0.25, 0.2]
        })
        fig = go.Figure()
        for param in data['param'].unique():
            param_data = data[data['param'] == param]
            fig.add_trace(go.Scatter(x=param_data['value'], y=param_data['loss'], mode='lines+markers', name=param))
        fig.update_layout(title='Hyperparameter Tuning Results', xaxis_title='Value', yaxis_title='Loss')
        return fig
    return go.Figure()

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
%%writefile real_time_visualization.py
import pandas as pd
import plotly.graph_objs as go
from dash import dcc, html, Input, Output
import dash
import time
import threading

app = dash.Dash(__name__)

data = pd.DataFrame(columns=['time', 'price'])

def fetch_real_time_data():
    global data
    while True:
        new_data = {'time': pd.Timestamp.now(), 'price': pd.np.random.rand()}
        data = data.append(new_data, ignore_index=True)
        time.sleep(1)

threading.Thread(target=fetch_real_time_data).start()

app.layout = html.Div([
    html.H3('Real-Time Market Analysis'),
    dcc.Graph(id='market-analysis-chart'),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # in milliseconds (update every 1 second)
        n_intervals=0
    )
])

@app.callback(
    Output('market-analysis-chart', 'figure'),
    [Input('interval-component', 'n_intervals')])
def update_market_analysis(n):
    fig = go.Figure(data=[go.Scatter(x=data['time'], y=data['price'], mode='lines+markers')])
    fig.update_layout(title='Real-Time Market Data', xaxis_title='Time', yaxis_title='Price')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
%%writefile performance_reporting.py
import pandas as pd
import plotly.graph_objs as go
from dash import dcc, html, Input, Output, State
import dash

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H3('Performance Report Generation'),
    dcc.Dropdown(
        id='report-period',
        options=[
            {'label': 'Last Day', 'value': '1D'},
            {'label': 'Last Week', 'value': '1W'},
            {'label': 'Last Month', 'value': '1M'}
        ],
        value='1D'
    ),
    html.Button('Generate Report', id='generate-report-button', n_clicks=0),
    html.Div(id='report-output'),
    dcc.Graph(id='performance-metrics-graph')
])

@app.callback(
    [Output('report-output', 'children'), Output('performance-metrics-graph', 'figure')],
    [Input('generate-report-button', 'n_clicks')],
    [State('report-period', 'value')])
def generate_performance_report(n_clicks, period):
    if n_clicks > 0:
        report_data = pd.DataFrame({
            'metric': ['P&L', 'Sharpe Ratio', 'Win Rate'],
            'value': [1000, 1.5, 0.6]
        })
        report_text = f"Performance Report for Period: {period}\n"
        report_text += report_data.to_string(index=False)

        fig = go.Figure(data=[
            go.Bar(x=report_data['metric'], y=report_data['value'])
        ])
        fig.update_layout(title='Performance Metrics')

        return report_text, fig
    return "", go.Figure()

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
%%writefile dashboard_customization.py
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import plotly.graph_objs as go

app = dash.Dash(__name__)

available_metrics = ['P&L', 'Sharpe Ratio', 'Win Rate', 'Volatility', 'Drawdown']

app.layout = html.Div([
    html.H3('Customize Dashboard'),
    dcc.Checklist(
        id='selected-metrics',
        options=[{'label': metric, 'value': metric} for metric in available_metrics],
        value=['P&L', 'Sharpe Ratio']
    ),
    html.Button('Save Configuration', id='save-config-button', n_clicks=0),
    html.Div(id='config-status'),
    dcc.Graph(id='custom-dashboard')
])

@app.callback(
    Output('config-status', 'children'),
    Input('save-config-button', 'n_clicks'),
    State('selected-metrics', 'value'))
def save_configuration(n_clicks, selected_metrics):
    if n_clicks > 0:
        return f"Configuration saved: {', '.join(selected_metrics)}"
    return ""

@app.callback(
    Output('custom-dashboard', 'figure'),
    Input('selected-metrics', 'value'))
def update_dashboard(selected_metrics):
    data = pd.DataFrame({
        'metric': selected_metrics,
        'value': [pd.np.random.rand() * 1000 for _ in selected_metrics]
    })
    fig = go.Figure(data=[
        go.Bar(x=data['metric'], y=data['value'])
    ])
    fig.update_layout(title='Custom Dashboard')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
%%writefile train_pipeline.py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from preprocessing import preprocess_data, create_lstm_dataset
from genetic_optimization import genetic_algorithm_optimization

# Load and preprocess data
data = pd.read_csv('historical_data.csv')
preprocessed_data, scaler = preprocess_data(data)

# Prepare LSTM dataset
scaled_close = preprocessed_data['scaled_close'].values.reshape(-1, 1)
X, y = create_lstm_dataset(scaled_close)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Optimize hyperparameters using Genetic Algorithm
best_params = genetic_algorithm_optimization()

# Build and train the LSTM model
def build_lstm_model(input_shape, units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(input_shape, 1)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = build_lstm_model(input_shape=X_train.shape[1], units=best_params[0], dropout_rate=best_params[1])
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), verbose=1)
model.save('trained_lstm_model.h5')

In [ ]:
%%writefile genetic_optimization.py
import numpy as np
from deap import base, creator, tools, algorithms
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

# Define the genetic algorithm parameters
POPULATION_SIZE = 50
GENERATIONS = 30
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.5

def build_lstm_model(input_shape, units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(input_shape, 1)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def evaluate_model(individual):
    units, dropout_rate = individual
    model = build_lstm_model(input_shape=X_train.shape[1], units=units, dropout_rate=dropout_rate)
    model.fit(X_train, y_train, epochs=10, verbose=0)
    loss = model.evaluate(X_val, y_val, verbose=0)
    return loss,

# DEAP setup
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_int", np.random.randint, 10, 100)
toolbox.register("attr_float", np.random.uniform, 0.1, 0.5)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_int, toolbox.attr_float), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate_model)

# Genetic Algorithm
def genetic_algorithm_optimization():
    population = toolbox.population(n=POPULATION_SIZE)
    algorithms.eaSimple(population, toolbox, cxpb=CROSSOVER_RATE, mutpb=MUTATION_RATE, ngen=GENERATIONS, verbose=True)
    best_individual = tools.selBest(population, k=1)[0]
    return best_individual

In [ ]:
%%writefile prediction_pipeline.py
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from preprocessing import preprocess_data, create_lstm_dataset

# Load trained model
model = load_model('trained_lstm_model.h5')

def generate_predictions(data):
    preprocessed_data, scaler = preprocess_data(data)
    scaled_close = preprocessed_data['scaled_close'].values.reshape(-1, 1)
    X, y = create_lstm_dataset(scaled_close)
    predictions = model.predict(X)
    predictions = scaler.inverse_transform(predictions)
    return predictions

In [ ]:
%%writefile Dockerfile
# Use the official lightweight Python image.
# https://hub.docker.com/_/python
FROM python:3.9-slim

# Copy local code to the container image.
ENV APP_HOME /app
WORKDIR $APP_HOME
COPY . ./

# Install production dependencies.
RUN pip install --no-cache-dir -r requirements.txt

# Run the web service on container startup.
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]

In [ ]:
%%writefile requirements.txt
alpaca-trade-api
alpha_vantage
tensorflow
scikit-learn
matplotlib
dash
dash-core-components
dash-html-components
dash-bootstrap-components
fastapi
uvicorn
nest-asyncio
pyngrok
transformers
plotly
pandas
requests
backtrader
boto3
keras-tuner
deap

In [ ]:
!docker build -t autogpt-trader .
!docker run -p 8000:8000 autogpt-trader

In [ ]:
!python data_streams.py  # This will start the real-time data fetching

In [ ]:
import uvicorn
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
import nest_asyncio
from pyngrok import ngrok
from threading import Thread
from visualization import run_dash, dash_app
from trade_execution import execute_trade
from preprocessing import preprocess_data
from model_preparation import build_lstm_model

nest_asyncio.apply()
app = FastAPI()

@app.get("/")
def read_root():
    html_content = """
    <html>
        <head>
            <title>AutoGPT Trader Dashboard</title>
        </head>
        <body>
            <iframe src="/dash/" style="width:100%; height:100vh;"></iframe>
        </body>
    </html>
    """
    return HTMLResponse(content=html_content)

@app.post("/predict/")
def predict(data: dict):
    preprocessed_data, scaler = preprocess_data(data)
    X, y = create_lstm_dataset(preprocessed_data['scaled_close'].values.reshape(-1, 1))
    lstm_model = build_lstm_model(X.shape[1])
    predictions = lstm_model.predict(X)
    return {"predictions": scaler.inverse_transform(predictions).tolist()}

@app.get("/dash/")
def get_dash_app(request: Request):
    return HTMLResponse(dash_app.index(), status_code=200)

ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)

if __name__ == "__main__":
    dash_thread = Thread(target=run_dash)
    dash_thread.start()
    uvicorn.run(app, host="0.0.0.0", port=8000, loop="asyncio")

In [ ]:
import time
from threading import Thread

def monitor_ngrok_tunnel():
    while True:
        tunnels = ngrok.get_tunnels()
        if not tunnels:
            print("No active tunnels found. Reconnecting...")
            ngrok_tunnel = ngrok.connect(name="myservice", config_path=ngrok_config_path)
            print("Reconnected. Public URL:", ngrok_tunnel.public_url)
        else:
            print("Tunnel is active:", tunnels[0].public_url)
        time.sleep(60)

# Start monitoring in a separate thread
monitor_thread = Thread(target=monitor_ngrok_tunnel)
monitor_thread.start()